In [6]:
#importing important libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV
import tensorflow
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import SimpleRNN,LSTM,Dense,Dropout
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [7]:
#importing dataset
df = pd.read_csv("C:/Users/megal/Downloads/AAPL.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.407747,117258400.0
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.386473,43971200.0
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.358108,26432000.0
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366972,21610400.0
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.377609,18362400.0


In [28]:
#checking there is any null values
df.isnull().sum()

Date         0
Open         1
High         1
Low          1
Close        1
Adj Close    1
Volume       1
dtype: int64

In [8]:
# Dropping the null values
df.dropna(subset = ['Open'],inplace = True)

In [ ]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [9]:
#converting date column to datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)

In [10]:
y = df['Close'].values

In [11]:
#scaling
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1,1))

In [12]:
#creating sequence
step = 3
def create_sequence(df,step):
    x,y = [],[]
    for i in range(len(df) - step):
        x.append(df[i:i + step])
        y.append(df[i + step])
    return np.array(x),np.array(y)

In [13]:
x,y = create_sequence(y_scaled,step)

In [15]:
#spliting the data into training and testing sets
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 5)

In [16]:
# Reshape x_train and x_test for LSTM input
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [15]:
#building the simple rnn model
model = Sequential([
    layers.SimpleRNN(32,input_shape = (x_train.shape[1],x_train.shape[2])),
    layers.Dense(128,activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

In [16]:
#compile the model
model.compile(optimizer = Adam(learning_rate = 0.001),loss = 'mean_squared_error',metrics = ['mean_squared_error'])

In [17]:
#earlying stopping
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 3,restore_best_weights = True)

#training the model
history_rnn = model.fit(x_train,y_train,epochs = 20,batch_size = 32,callbacks = [early_stopping])

Epoch 1/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0183 - mean_squared_error: 0.0183
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 7/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 8/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 9/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 10/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 11/20
246/246 ━━━━━━━━━━━━━━━━━━━

In [16]:
import pickle as pkl
with open('simple_rnn.pkl','wb') as file:
    pkl.dump(model,file)

In [17]:
with open('simple_rnn.pkl','rb') as file:
    simple = pkl.load(file)

In [19]:
testing = simple.evaluate(x_test,y_test)

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.7250e-05 - mean_squared_error: 3.7250e-05


In [17]:
#building the LSTM model
model = Sequential([
    layers.LSTM(64),
    layers.Dense(128,activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

In [18]:
#compiling the model
model.compile(optimizer = Adam(learning_rate = 0.001),loss = 'mean_squared_error',metrics = ['accuracy'])

In [19]:
#earlying stopping
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 3,restore_best_weights = True)

#training the LSTM model
history_lstm = model.fit(x_train,y_train,epochs = 20,batch_size=32,callbacks = [early_stopping])

Epoch 1/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 2.5393e-05 - loss: 0.0088
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 7.6224e-05 - loss: 0.0014
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 7.9143e-06 - loss: 0.0014
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 2.2291e-04 - loss: 0.0013
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.3301e-04 - loss: 0.0014
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 3.0492e-05 - loss: 0.0015
Epoch 7/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 2.3532e-05 - loss: 0.0012
Epoch 8/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 3.3787e-05 - loss: 0.0013
Epoch 9/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0551e-04 - loss: 0.0013
Epoch 10/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.6253e-04 - loss: 0.0013
Epoch 11/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0318e-04 - loss: 0.0013
Epoch 12

In [23]:
with open('lstm.pkl','wb') as file:
    pkl.dump(model,file)

In [24]:
with open('lstm.pkl','rb') as file:
    lstm = pkl.load(file)

In [28]:
y_pred = lstm.predict(x_test)

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [30]:
mse = mean_squared_error(y_test,y_pred)
print(f'Mean Squared Error : {mse}')

Mean Squared Error : 0.00019668047414085808


In [ ]:
# Make predictions on the test set
y_pred = model.predict(x_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

In [25]:
test = lstm.evaluate(x_test,y_test)

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.8681e-04 - mean_squared_error: 1.8681e-04 
